# TM10007 Assignment template

In [17]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [1]:
# Import functions
from brats.load_data import load_data
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
import pandas as pd 

In [2]:
# Import data
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 167
The number of columns: 725


In [3]:
remove_features = data.T[data.isna().sum(axis=0) < 130]
removed_features = remove_features.T
print(f'The number of samples: {len(removed_features)}')
print(f'The number of columns: {len(removed_features.columns)}')

remove_patients= removed_features[removed_features.isna().sum(axis=1) < 400]
print(f'The number of samples: {len(remove_patients)}')
print(f'The number of columns: {len(remove_patients.columns)}')

The number of samples: 167
The number of columns: 705
The number of samples: 167
The number of columns: 705


In [4]:
# replace NaNs
# clean_data = KNNImputer(n_neighbors=10)
# clean_data.fit_transform(features)

# without_labels = remove_patients.drop('label', axis=1)
# without_labels.replace(r'#DIV/0!', 0)
# clean_data = without_labels.replace(np.NaN, without_labels.mean(axis = 0))
# clean_data.isna().sum().sum()

print(remove_patients.isna().sum().sum())
new = remove_patients.replace(r'#DIV/0!', 0, regex=True)
print(new.isna().sum().sum())
clean_data = new.replace(np.NaN, remove_patients.mean(axis = 0))
print(clean_data.isna().sum().sum())



4662
4662
0


In [5]:
# Extract labels and features
label = clean_data['label'].to_numpy()
features = clean_data.drop('label', axis=1).to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'reset_index'

In [7]:
def our_pca(data_train, data_test, label_train, label_test, j=20):
    """
    This function calculates the PCA for j principal components
    """

    #Standard Scaling
    scaler = StandardScaler().fit(data_train)
    data_train = scaler.transform(data_train)
    data_test = scaler.transform(data_test)

    #Train on trainingsdata and apply PCA
    pca = PCA(n_components=j).fit(data_train)
    train_array = pca.transform(data_train)
    test_array = pca.transform(data_test)

    #Transform to numpy and from array to float
    label_train_array = label_train.squeeze().astype(np.int)
    label_test_array = label_test.squeeze().astype(np.int)

    return(train_array, test_array, label_train_array, label_test_array)


In [8]:
# Split data in training en test
rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=36851234)

for train_index, test_index in rskf.split(features, label):
    print("TRAIN:", train_index, "TEST:", test_index)
    data_train, data_test = features[train_index], features[test_index]
    label_train, label_test = label[train_index], label[test_index]
    train_array, test_array, label_train_array, label_test_array = our_pca(data_train, data_test, label_train, label_test)

# PCA op training en test


TRAIN: [  0   1   2   3   8   9  10  15  19  20  23  24  25  26  27  29  33  34
  38  39  40  41  44  47  48  50  51  52  58  59  60  62  65  66  67  69
  70  74  75  78  80  81  82  83  86  87  89  91  96  98 101 106 108 109
 111 112 113 114 115 117 119 120 121 122 123 124 129 130 132 134 139 142
 144 149 151 152 154 155 159 161 162 163 166] TEST: [  4   5   6   7  11  12  13  14  16  17  18  21  22  28  30  31  32  35
  36  37  42  43  45  46  49  53  54  55  56  57  61  63  64  68  71  72
  73  76  77  79  84  85  88  90  92  93  94  95  97  99 100 102 103 104
 105 107 110 116 118 125 126 127 128 131 133 135 136 137 138 140 141 143
 145 146 147 148 150 153 156 157 158 160 164 165]


ValueError: Input contains infinity or a value too large for dtype('float64').